In [1]:
#%matplotlib inline
import pandas as pd
from pandas import DataFrame
from pandas.io import sql
from pandas.tseries.offsets import *
import numpy as np
from pprint import pprint
# import pylab as plt
import pymysql
# import seaborn
# from matplotlib import pyplot
from datetime import datetime,timedelta
import csv
import math
pd.options.display.float_format = '{:,.2f}'.format
import json
from IPython.display import display, HTML

In [2]:
with open('config.json') as f:
    conf = json.load(f)

In [3]:
con = pymysql.connect(host=conf['host'], user=conf['user'], passwd=conf['password'], db=conf['database'])

In [4]:
def get_trades(year):
    """
    return all trades from start of year
    :param year: the year to get trades
    :return: DataFrame of all trades of year
    """

    start_date = '{}-01-01'.format(year)
    
    query = """
        SELECT a.code, a.fundCode, a.orderType, a.side, a.swap, a.tradeDate, a.PB, 
              g.commrate as gcomrate, d.brokerCommissionRate,
              # a.commission,
              (@commInUSD := a.commission*f.rate) AS commInUSD,
              (@commrate := IF (g.commrate IS NOT NULL, ROUND(g.commrate,4), ROUND(d.brokerCommissionRate,4))) AS CommRate,
              (@jpresearch := IF(b.currencyCode IN ("JPY", "USD") AND c.instrumentType="EQ" AND  (@commrate<> 0.0004), 
                  @commInUSD*11/15,0)*1.0) AS JPResearch,
                  IF(b.currencyCode IN ("JPY", "USD") AND c.instrumentType="EQ" AND (@commrate=0.0004), 
                      @commInUSD, 
                      0)*1.0 AS JPDis,
              (@clearing:=
              IF (c.instrumentType IN ("FU", "OP"), 
                  IF(SUBSTRING(a.code, 1, 2) IN ("TP", "NK"), 500, 
                    IF(SUBSTRING(a.code, 1, 2)="JP",50, 
                    IF(SUBSTRING(a.code, 1, 2) IN ("HC", "HI"), 30, 0) 
                )
               ) * a.quantity *f.rate, 0  )) AS Clearing,
              IF(b.currencyCode IN ("JPY", "USD")  AND c.instrumentType="EQ" AND (@commrate=0.0004 OR @commrate=0),
                  0,
                  IF(b.currencyCode IN ("JPY", "USD")  AND c.instrumentType="EQ",
                  @commInUSD - @jpresearch,
                  IF(b.currencyCode="JPY" AND c.instrumentType IN ("FU", "OP"), 
                  @commInUSD - @clearing,0) )) AS JPExec,
              d.brokerCode, c.instrumentType, b.currencyCode,
              IF(d.brokerCode="BXS", "Soft", IF(d.brokerCode="INSH", "Nomura",e.name)) AS brokerName,
              (@tax := CASE CONCAT(c.instrumentType, a.orderType, b.currencyCode)
                        WHEN "EQBCNY" THEN 0.000098
                        WHEN "EQSCNY" THEN 0.001098
                        WHEN "EQBHKD" THEN 0.00108
                        WHEN "EQSHKD" THEN 0.00108
                        WHEN "EQSTWD" THEN 0.003
                        WHEN "EQSKRW" THEN 0.003
                        WHEN "EQBSGD" THEN 0.0004
                        WHEN "EQSSGD" THEN 0.0004
                        ELSE 0
                        END
              ) AS tax,
              (@asiadeal := IF (b.currencyCode  NOT IN ("JPY", "USD")  AND c.instrumentType="EQ" AND d.brokerCommissionRate > 0.01,
                                a.gross * f.rate * (d.brokerCommissionRate - @tax ), 0)) AS asiaDeal,
              (@asiaResearch := IF(b.currencyCode  NOT IN ("JPY", "USD")  AND c.instrumentType="EQ" AND @asiadeal=0, 
                IF(d.brokerCommissionRate-@tax-0.0005>= 0, d.brokerCommissionRate-@tax-0.0005, 0) *f.rate*a.gross, 0)) AS asiaResearch,
              IF (b.currencyCode NOT  IN ("JPY", "USD")  AND c.instrumentType="EQ" AND @asiadeal=0,
                0.0005 * f.rate * a.gross,
                IF (b.currencyCode NOT IN ("JPY", "USD")  AND c.instrumentType IN ("FU", "OP"), @commInUSD-@clearing, 0)
              ) AS asiaExecution,
              IF (a.swap="SWAP", @asiaResearch, 0)*1.0 AS HCSwaps
            FROM t08Reconcile a
              INNER JOIN t01Instrument c ON a.code = c.quick
              INNER JOIN t02Currency b ON b.currencyID = c.currencyID
              INNER JOIN t08Reconcile d ON a.matchDoric = d.primaryID # a.primaryID = d.matchBrokers AND d.srcFlag ="D"
              INNER JOIN t06DailyCrossRate f ON f.priceDate=a.tradeDate AND f.base=b.currencyCode AND f.quote="USD"
              LEFT JOIN t02Broker e ON e.brokerCode = a.brokerCode
              left join (select a.code, a.orderType, a.side, a.swap, a.tradeDate, a.settleDate, a.brokerCode, MAX(a.brokerCommissionRate) as commrate
            from t08Reconcile a
            where a.status="A" and a.srcFlag="D" and a.tradeDate >= {}
            group by a.code, a.orderType, a.side, a.swap, a.tradeDate, a.settleDate, a.brokerCode
              ) g ON a.code=g.code and a.orderType=g.orderType and a.side=g.side and a.swap=g.swap and a.tradeDate=g.tradeDate
                     and a.settleDate=g.settleDate  and d.brokerCode=g.brokerCode
            WHERE a.tradeDate>='{}' AND a.srcFlag="K"
            ORDER BY a.tradeDate, a.code;
        """.format(start_date, start_date)
    
#     print(query)

    ytd_trades = sql.read_sql(query, con, parse_dates=['tradeDate'], index_col='tradeDate')
    return ytd_trades

In [5]:
def get_quarter_trades(year, quarter, ytd_trades):
    if quarter == 1:
        start = '{}-01-01'.format(year)
        end = '{}-03-31'.format(year)
    elif quarter == 2:
        start = '{}-04-01'.format(year)
        end = '{}-06-30'.format(year)
    elif quarter == 3:
        start = '{}-07-01'.format(year)
        end = '{}-09-30'.format(year)
    elif quarter == 4:
        start = '{}-10-01'.format(year)
        end = '{}-12-31'.format(year)
    else:
        return None
    return ytd_trades[start:end]

In [6]:
def format_2f(df):
    t = df.copy()
    columns = ['JPResearch', 'JPExec', 'JPDis', 'res_target', 'balance_usd']
    t[columns] = t[columns].applymap(lambda x: '$ {:12,.0f}'.format(x) if x > 0 else '')
    t['balance_jpy'] = t['balance_jpy'].apply(lambda x: '¥ {:12,.0f}'.format(x) if x > 0 else '')
    t['rank'] = t['rank'].apply(lambda x: '{:.0f}'.format(x) if not np.isnan(x) else '')
    t['research'] = t['research'].apply(lambda x: '{:5.2f}%'.format(x) if x > 0 else '')
    t['accrued'] = t['accrued'].apply(lambda x: '{:5.0f}%'.format(x) if not np.isnan(x) else '')
    t['exec_target'] = t['exec_target'].apply(lambda x: '{:5.0f}%'.format(x) if x > 0 else '')
    return t

In [7]:
def calculate_commission(quarter_trades, jp_ranks_df, jp_quarter_commission_budget, usd_jpy):
    table = (quarter_trades
             .groupby(['brokerName', 'currencyCode'])
             .sum()
             .loc[(slice(None), ['JPY', 'USD']), ['JPResearch', 'JPExec', 'JPDis']]
             .groupby(level=0).sum()
             .reset_index()
             .set_index('brokerName')
             .merge(jp_ranks_df, how='right', left_index=True,
                    right_index=True)  # some names removed like Barclays and Softs
             .fillna(0)
             .sort_values(by='rank', axis=0)
             .assign(res_target=lambda df: df['research'] * jp_quarter_commission_budget / 100)
             .assign(balance_usd=lambda df: df['res_target'] - df['JPResearch'])
             .assign(balance_jpy=lambda df: df['balance_usd'] * usd_jpy)
             .assign(accrued=lambda df: (df['JPExec'] + df['JPDis']) * 100 / (df['JPExec'].sum() + df['JPDis'].sum()))
             .reset_index()
             .set_index('rank')
             )

    exec_target = [11, 11, 10, 10, 10, 7, 7, 7, 7, 7, 3, 3, 3, 2, 1]
    if len(exec_target) < len(table.index):
        exec_target = exec_target + [0] * (len(table.index) - len(exec_target))
    table['exec_target'] = pd.Series(exec_target, index=table.index)
    return (table
            .reset_index()
            .append(table[['research', 'res_target', 'JPResearch', 'JPExec',
                           'JPDis', 'balance_usd', 'balance_jpy', 'exec_target']].sum(), ignore_index=True)
            .pipe(format_2f)
            [['rank', 'brokers', 'research', 'res_target', 'JPResearch', 'JPExec',
              'JPDis', 'balance_usd', 'balance_jpy', 'exec_target', 'accrued']]
            )

In [8]:
date = '2016-06-30'

if date is None:
    date = datetime.date.today()
else:
    try:
        date = datetime.strptime(date, '%Y-%m-%d')
    except ValueError:
        date = datetime.date.today()

quarter = (date.month-1) // 3 + 1

ytd_trades = get_trades(date.year)

/Users/cuong/anaconda/envs/attribution/lib/python2.7/site-packages/pymysql/cursors.py:146: Warning: Incorrect date value: '2014' for column 'tradeDate' at row 1
  result = self._query(query)


In [9]:
date.year, quarter

(2016, 2)

In [10]:
ytd_trades.head()

,code,fundCode,orderType,side,swap,PB,gcomrate,brokerCommissionRate,commInUSD,CommRate,...,JPExec,brokerCode,instrumentType,currencyCode,brokerName,tax,asiaDeal,asiaResearch,asiaExecution,HCSwaps
tradeDate,,,,,,,,,,,,,,,,,,,,,
2016-01-04,1138 HK,04F08910,S,L,,MS,0.00,0.00,882.72,0.00,...,0.00,CSFH,EQ,HKD,CS,0.00,0.00,429.47,143.44,0.00
2016-01-04,1138 HK,PLOJ2010,S,L,,MS,0.00,0.00,203.77,0.00,...,0.00,CSFH,EQ,HKD,CS,0.00,0.00,99.17,33.10,0.00
2016-01-04,1138 HK,04F08900,S,L,,MS,0.00,0.00,"5,768.18",0.00,...,0.00,CSFH,EQ,HKD,CS,0.00,0.00,"2,806.24",937.29,0.00
2016-01-04,1338 TT,04F08910,S,L,,MS,0.00,0.00,213.96,0.00,...,0.00,KGII,EQ,TWD,KGI Securities,0.00,0.00,47.55,23.77,0.00
2016-01-04,1338 TT,04F08900,S,L,,MS,0.00,0.00,"1,547.09",0.00,...,0.00,KGII,EQ,TWD,KGI Securities,0.00,0.00,343.80,171.90,0.00


In [11]:
# Asia commission budget is 2mm  USD per year
# Japan has gone through a few iterations but it's basically approximately 7.5 mm USD per year
jp_annual_commission_budget = 7500000  # usd
asia_annual_commission_budget = 2000000  # usd
jp_quarter_commission_budget = jp_annual_commission_budget / 4.0
asia_quarter_commission_budget = asia_annual_commission_budget / 4.0

jp_quarter_commission_budget = 10031527/4.0  # to make it same with old system 2016Q1

In [12]:
# broker rank for 2016Q2
data = {
    'brokers': ['BAML', 'Mizuho Securities', 'Nomura', 'Japan Equity Research',
                'Citi', 'Mitsubishi UFJ', 'Tokai', 'Ichiyoshi', 'SMBC Nikko',
                'BNP', 'Deutsche', 'CLSA', 'Daiwa', 'Jefferies',
                'CS', 'UBS', 'JP Morgan', 'Goldman Sachs', 'Okasan',
                'MS', 'Macquarie'
                ],
    'rank': [6, 2, 1, 17,
             4, 10, 15, 14, 5,
             13, 18, 21, 8, 16,
             9, 12, 7, 3, 19,
             11, 20],
    'research': map(lambda x: x, [6.76, 10.04, 12.52, 0.97,
                                  8.53, 5.84, 1.18, 2.29, 7.56,
                                  2.41, 0.88, 0.25, 6.47, 1.09,
                                  5.90, 8.91, 6.61, 8.94, 0.85,
                                  5.32, 0.25
                                  ])
}

jp_ranks_df = pd.DataFrame(data).set_index('brokers')
usd_jpy = 112.27  # 2016Q1
hkd_jpy = 7.754
usd_jpy = 102.66  # 2016Q2
hkd_jpy = 7.759
quarter_trades = get_quarter_trades(date.year, quarter, ytd_trades)

In [13]:
table = calculate_commission(quarter_trades, jp_ranks_df, jp_quarter_commission_budget, usd_jpy)
qtd_clearing = "{:,.0f}".format(quarter_trades['Clearing'].sum())
ytd_clearing = "{:,.0f}".format(ytd_trades['Clearing'].sum())

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
mon1, mon2, mon3 = {
    1: months[0:3],
    2: months[3:6],
    3: months[6:9],
    4: months[9:]
}[quarter]

if quarter == 1:
    m1, m2, m3 = [1, 2, 3]
elif quarter == 2:
    m1, m2, m3 = [4, 5, 6]

mon1_trades = quarter_trades['{}-{:02d}'.format(date.year, m1)]
soft1 = "{:,.0f}".format(mon1_trades[(mon1_trades['brokerName'] == 'Soft')]['JPResearch'].sum())
soft_ms1 = "{:,.0f}".format(mon1_trades[mon1_trades['brokerName'] == 'Soft']['JPExec'].sum())

mon2_trades = quarter_trades['{}-{:02d}'.format(date.year, m2)]
soft2 = "{:,.0f}".format(mon2_trades[(mon2_trades['brokerName'] == 'Soft')]['JPResearch'].sum())
soft_ms2 = "{:,.0f}".format(mon2_trades[mon2_trades['brokerName'] == 'Soft']['JPExec'].sum())

mon3_trades = quarter_trades['{}-{:02d}'.format(date.year, m3)]
soft3 = "{:,.0f}".format(mon3_trades[(mon3_trades['brokerName'] == 'Soft')]['JPResearch'].sum())
soft_ms3 = "{:,.0f}".format(mon3_trades[mon3_trades['brokerName'] == 'Soft']['JPExec'].sum())

soft_qtd = '{:,.0f}'.format(quarter_trades[(quarter_trades['brokerName'] == 'Soft') &
                          (quarter_trades['currencyCode'] == 'JPY')]['JPResearch'].sum())
soft_msqtd = '{:,.0f}'.format(quarter_trades[(quarter_trades['brokerName'] == 'Soft') &
                          (quarter_trades['currencyCode'] == 'JPY')]['JPExec'].sum())

In [14]:
table

,rank,brokers,research,res_target,JPResearch,JPExec,JPDis,balance_usd,balance_jpy,exec_target,accrued
0,1,Nomura,12.52%,"$ 313,987","$ 319,687","$ 116,250","$ 125,753",,,11%,11%
1,2,Mizuho Securities,10.04%,"$ 251,791","$ 256,774","$ 93,372","$ 137,763",,,11%,11%
2,3,Goldman Sachs,8.94%,"$ 224,205","$ 223,357","$ 81,221","$ 121,548",$ 848,"¥ 87,055",10%,10%
3,4,Citi,8.53%,"$ 213,922","$ 214,984","$ 78,176","$ 133,581",,,10%,10%
4,5,SMBC Nikko,7.56%,"$ 189,596","$ 193,577","$ 70,392","$ 133,193",,,10%,10%
5,6,BAML,6.76%,"$ 169,533","$ 159,546","$ 58,640","$ 90,844","$ 9,986","¥ 1,025,205",7%,7%
6,7,JP Morgan,6.61%,"$ 165,771","$ 164,270","$ 59,735","$ 93,597","$ 1,501","¥ 154,061",7%,7%
7,8,Daiwa,6.47%,"$ 162,260","$ 160,650","$ 58,418","$ 95,230","$ 1,610","¥ 165,260",7%,7%
8,9,CS,5.90%,"$ 147,965","$ 143,477","$ 52,173","$ 90,329","$ 4,488","¥ 460,752",7%,7%
9,10,Mitsubishi UFJ,5.84%,"$ 146,460","$ 147,925","$ 53,791","$ 85,298",,,7%,7%


In [15]:
currency_mask = ytd_trades['currencyCode'] == 'JPY'
(ytd_trades.groupby(['brokerName', currency_mask])
           .sum()
           .assign(AsiaYTD = lambda x : x.asiaResearch + x.asiaExecution)
           .unstack()
           .fillna(0) [['JPResearch', 'JPExec', 'Clearing', 'commInUSD'
                        , 'AsiaYTD'
                       ]]
           .rename(columns={True: 'JP', False: 'Asia'})
         #.loc[(slice(None), 'JPY'), ['JPResearch', 'JPExec', 'commInUSD']]
)

JPResearch                JPExec             \
currencyCode                           Asia           JP     Asia         JP   
brokerName                                                                     
BAML                               4,615.56   855,423.95 1,678.38 382,597.24   
BNP                                    0.00   263,440.88     0.00  95,796.68   
Barclays                               0.00     5,991.92     0.00   2,178.88   
CLSA                                   0.00    18,277.65     0.00   6,646.42   
CS                                     0.00   741,112.06     0.00 269,495.29   
Cantor                                 0.00         0.00     0.00       0.00   
Citi                                   0.00   829,053.04     0.00 305,253.45   
Daiwa                                  0.00   992,860.20     0.00 361,040.07   
Deutsche                               0.00   139,752.17     0.00  50,818.97   
Fubon Securities                       0.00         0.00     0.00       0.00   
Goldman Sachs                          0.00   853,809.30     0.00 316,605.61   
Ichiyoshi                              0.00   206,108.78     0.00  74,948.65   
JP Morgan                              0.00   794,311.41     0.00 288,840.51   
Jefferies                              0.00   214,906.44     0.00  78,147.80   
KGI Securities                         0.00         0.00     0.00       0.00   
MS                                26,404.49   685,441.91 9,601.63 374,418.76   
Macquarie                              0.00   191,737.78     0.00  69,722.83   
Mitsubishi UFJ                         0.00   894,414.60     0.00 325,241.67   
Mizuho Securities                      0.00 1,272,644.60     0.00 462,779.85   
Nomura                                 0.00 1,351,275.52     0.00 491,372.92   
Okasan                                 0.00   142,266.39     0.00  51,733.23   
RHB OSK Securities Hong Kong Lim       0.00         0.00     0.00       0.00   
SMBC Nikko                             0.00 1,297,491.34     0.00 471,815.03   
Soft                                   0.00   917,504.54     0.00 333,638.02   
UBS                                    0.00 1,174,380.91     0.00 427,047.61   
Yuanta                                 0.00         0.00     0.00       0.00   

                                 Clearing             commInUSD               \
currencyCode                         Asia         JP       Asia           JP   
brokerName                                                                     
BAML                                 0.00  71,533.97 680,744.37 1,650,707.84   
BNP                                  0.00       0.00       0.00   438,803.84   
Barclays                             0.00       0.00       0.00     8,170.81   
CLSA                                 0.00       0.00  97,471.50    24,924.07   
CS                                   0.00       0.00 539,157.54 1,332,120.96   
Cantor                               0.00       0.00 143,682.91         0.00   
Citi                                 0.00   3,779.61 504,991.27 1,492,744.65   
Daiwa                                0.00       0.00 169,345.81 1,915,948.56   
Deutsche                             0.00       0.00 151,871.63   191,692.04   
Fubon Securities                     0.00       0.00 231,301.60         0.00   
Goldman Sachs                        0.00  11,196.09 171,666.57 1,510,405.95   
Ichiyoshi                            0.00       0.00       0.00   281,057.43   
JP Morgan                            0.00       0.00 384,076.54 1,502,556.44   
Jefferies                            0.00       0.00 181,365.39   293,054.24   
KGI Securities                       0.00       0.00 358,835.26         0.00   
MS                               7,115.98 137,439.40 418,829.41 1,333,259.37   
Macquarie                            0.00       0.00 160,046.41   341,112.76   
Mitsubishi UFJ                       0.00       0.00       0.00 1,647,937.46   
Mizuho Securities               

In [16]:
# (ytd_trades.groupby(['brokerName', 'currencyCode'])
#            .sum()
#            #.assign(AsiaYTD = lambda x : x.asiaResearch + x.asiaExecution)
#            .unstack()
#            .fillna(0) [['commInUSD', 'JPResearch', 'JPExec', 'Clearing'
#             #            , 'AsiaYTD'
#                        ]]
#            .rename(columns={True: 'JP', False: 'Asia'})
#          #.loc[(slice(None), 'JPY'), ['JPResearch', 'JPExec', 'commInUSD']]
# )

In [17]:
# broker rank for each quarter - 2016Q1
data = {
    'brokers': ['BAML', 'Mizuho Securities',  'Nomura', 'Japan Equity Research', 
                'Citi', 'Mitsubishi UFJ', 'Tokai', 'Ichiyoshi', 'SMBC Nikko', 
                'BNP', 'Deutsche', 'CLSA', 'Daiwa', 'Jefferies', 
                'CS', 'UBS', 'JP Morgan', 'Goldman Sachs', 'Okasan', 
                'MS', 'Macquarie', 'Barclays'
               ],
    'rank': [6, 1, 2, 20, 9, 4, 18, 16, 3, 14, 19, 22, 5, 15, 11, 10, 12, 7, 17, 8, 21, 13],
    'balance': [2240, -664, -808, 0,
                58, 881, 0, 594, 691,
                1815, 1128, 1836, 1719, 2450,
                -8419, -417, 3744, 2854, 618,
                -2577, 928, 2348
               ],
    'research': map(lambda x: x, [7.55, 10.255114, 8.69, 0.81,
                 6.47, 7.95, 1.16, 1.36, 8.1,
                 2.19, 0.99, 0.13, 7.94, 1.46,
                 5.72, 8.18, 4.98, 6.99, 1.2,
                 6.86, 0.63, 2.35
                ])
}

jp_ranks_df = pd.DataFrame(data).set_index('brokers')
jp_ranks_df

,balance,rank,research
brokers,,,
BAML,2240,6,7.55
Mizuho Securities,-664,1,10.26
Nomura,-808,2,8.69
Japan Equity Research,0,20,0.81
Citi,58,9,6.47
Mitsubishi UFJ,881,4,7.95
Tokai,0,18,1.16
Ichiyoshi,594,16,1.36
SMBC Nikko,691,3,8.10


In [18]:
jp_quarter_commission_budget =10031527/4.0
jp_quarter_commission_budget

2507881.75

In [19]:
q1 = get_quarter_trades(2016, 1, ytd_trades)

For monthly balance need to divided res_target by 3 to find balance, but for quarter calculation, do not need to divide by 3.

In [20]:
# # debug duplicated trades for 2016Q1
# ubsw = q1[q1['brokerCode'] == 'UBSW']
# jan14 = ubsw.loc['2016-01-14']
# jan14[jan14['code'] == '6305']

In [21]:
## debug duplicated trades for 2016Q2
smbc = quarter_trades[quarter_trades['brokerCode'] == 'NIKK']
print(smbc['JPResearch'].sum())
smbc.loc['2016-05-17':'2016-05-17']['JPResearch'].sum() #[['code', 'fundCode', 'orderType', 'side', 'swap', 'JPResearch']]
smbc.loc['2016-05-17':'2016-05-17']#[['code', 'fundCode', 'orderType', 'side', 'swap', 'JPResearch']]

193576.949695


,code,fundCode,orderType,side,swap,PB,gcomrate,brokerCommissionRate,commInUSD,CommRate,...,JPExec,brokerCode,instrumentType,currencyCode,brokerName,tax,asiaDeal,asiaResearch,asiaExecution,HCSwaps
tradeDate,,,,,,,,,,,,,,,,,,,,,
2016-05-17,4348,04F08910,S,L,,CS,0.00,0.00,326.71,0.00,...,87.12,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00
2016-05-17,4348,PLOJ2010,S,L,,CS,0.00,0.00,49.00,0.00,...,13.07,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00
2016-05-17,4849,04F08900,B,L,,CS,0.00,0.00,138.56,0.00,...,36.95,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00
2016-05-17,4849,PLOJ2010,B,L,,CS,0.00,0.00,4.26,0.00,...,1.14,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00
2016-05-17,4849,04F08910,B,L,,CS,0.00,0.00,17.05,0.00,...,4.55,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00
2016-05-17,5471,04F08900,S,L,,MS,0.00,0.00,"1,134.25",0.00,...,302.47,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00
2016-05-17,5471,04F08910,S,L,,MS,0.00,0.00,147.30,0.00,...,39.28,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00
2016-05-17,5471,PLOJ2010,S,L,,MS,0.00,0.00,39.27,0.00,...,10.47,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00
2016-05-17,6300,04F08910,S,L,,CS,0.00,0.00,36.44,0.00,...,9.72,NIKK,EQ,JPY,SMBC Nikko,0.00,0.00,0.00,0.00,0.00


In [22]:
# q1[q1['brokerCode']=='MSUS']
print("QTD Clearing : {:,.0f}".format(q1['Clearing'].sum()))

QTD Clearing : 43,394


In [23]:
print("YTD Clearing : {:,.0f}".format(ytd_trades['Clearing'].sum()))

YTD Clearing : 231,065


In [24]:
print("Soft YTD: {:,.0f}".format(q1[(q1['brokerName'] == 'Soft') & (q1['currencyCode'] == 'JPY')]['JPResearch'].sum()))

Soft YTD: 174,208


In [25]:
jan = q1['2016-01']
print("Jan Soft= {:,.0f}".format(jan[(jan['brokerName'] == 'Soft') ]['JPResearch'].sum()))

Jan Soft= 55,915


In [26]:
jan_ms = jan[(jan['brokerName'] == 'Soft') ]['JPExec'].sum()
jan_ms
print('Soft MS: {:,.0f}'.format(jan_ms))

Soft MS: 20,333


In [27]:
(q1[q1['currencyCode'] != "JPY"]
   .groupby(['brokerName'])
   .sum()[['asiaResearch', 'asiaExecution', 'HCSwaps']]
   .assign(asiaYTD = lambda x: x.asiaResearch + x.asiaExecution)
   # ['commission', 'commInUSD', 'JPResearch', 'JPExec', 'Clearing']
)

,asiaResearch,asiaExecution,HCSwaps,asiaYTD
brokerName,,,,
BAML,"50,964.80","16,184.29","24,123.68","67,149.09"
CLSA,"40,724.07","11,994.31",0.00,"52,718.38"
CS,"71,288.16","19,842.69","61,683.38","91,130.84"
Cantor,"11,825.46","3,949.27",0.00,"15,774.73"
Citi,"33,476.77","9,021.41","29,481.44","42,498.18"
Daiwa,"13,391.83","3,414.55",0.00,"16,806.38"
Deutsche,"6,503.71","2,172.05",0.00,"8,675.76"
Fubon Securities,"17,025.33","8,512.67",0.00,"25,538.00"
Goldman Sachs,"16,182.94","5,404.57",0.00,"21,587.51"


In [28]:
(q1[q1['currencyCode'] != "JPY"]
 .groupby(['brokerName', 'currencyCode'])
 .sum()[['asiaResearch']]
 .unstack()
 .fillna(0)
 )

asiaResearch                            \
currencyCode                              CNY       HKD       KRW   SGD   
brokerName                                                                
BAML                                 1,423.24 40,976.62      0.00  0.00   
CLSA                                     0.00 21,592.20      0.00  0.00   
CS                                       0.00 24,059.66 20,278.24  0.00   
Cantor                                   0.00 11,825.46      0.00  0.00   
Citi                                     0.00 11,936.42      0.00  0.00   
Daiwa                                    0.00    794.25 12,597.58  0.00   
Deutsche                                 0.00  6,503.71      0.00  0.00   
Fubon Securities                         0.00      0.00      0.00  0.00   
Goldman Sachs                            0.00 16,182.94      0.00  0.00   
JP Morgan                                0.00 30,018.46  3,424.08  0.00   
Jefferies                                0.00 11,655.02      0.00  0.00   
KGI Securities                           0.00      0.00      0.00  0.00   
MS                                       0.00  7,285.95 19,863.43  0.00   
Macquarie                                0.00 16,684.25      0.00 60.05   
Nomura                                   0.00  6,726.81  9,115.28  0.00   
RHB OSK Securities Hong Kong Lim         0.00    181.91      0.00  0.00   
UBS                                      0.00 28,731.08  5,530.22  0.00   
Yuanta                                   0.00      0.00      0.00  0.00   

                                            
currencyCode                           TWD  
brokerName                                  
BAML                              8,564.93  
CLSA                             19,131.87  
CS                               26,950.26  
Cantor                                0.00  
Citi                             21,540.35  
Daiwa                                 0.00  
Deutsche                              0.00  
Fubon Securities                 17,025.33  
Goldman Sachs                         0.00  
JP Morgan                             0.00  
Jefferies                             0.00  
KGI Securities                   14,552.11  
MS                               10,194.55  
Macquarie                             0.00  
Nomura                           10,544.87  
RHB OSK Securities Hong Kong Lim      0.00  
UBS                                   0.00  
Yuanta                           13,554.17

In [29]:
# q2jpm = ytd_trades #.loc['2017-04-03':]  #,ytd_trades['brokerCode'] == 'JPMF']
# q2jpm[(q2jpm['brokerCode'] == 'RHBO') & (q2jpm['currencyCode'] != 'JPY')]

In [30]:
# from IPython.display import display, HTML
#HTML(q1[(q1['currencyCode'] == 'HKD') & (q1['brokerCode'] == 'MERT')].to_html())
#HTML(q1[(q1['currencyCode'] != 'JPY') & (q1['brokerName'] == 'Nomura')].to_html())
# q1[(q1['currencyCode'] != 'JPY') & (q1['brokerName'] == 'UBS')].count()